**Data Loading**

In [ ]:
from google.colab import drive
import zipfile
import pandas as pd
import numpy as np

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/time series project/best_track.csv')
df.head()

Mounted at /content/drive


,cyclone_id,datetime,flag,intensity,latitude,longitude,num_data_lines,pressure,revision_date,storm_name,time_interval,track_sequence,wind
0,CYC000001,1949011300,0,0,57,1399,49,1006,20110729.0,Carmen,6,1,0
1,CYC000001,1949011306,0,0,59,1393,49,1006,20110729.0,Carmen,6,2,0
2,CYC000001,1949011312,0,0,63,1387,49,1006,20110729.0,Carmen,6,3,0
3,CYC000001,1949011318,0,0,67,1380,49,1006,20110729.0,Carmen,6,4,0
4,CYC000001,1949011400,0,0,72,1373,49,1005,20110729.0,Carmen,6,5,0


**Comparing between geodesic distance/forward azimuth (bearing) And latitude/longitude**

geodesic distance/forward azimuth conversion

Latitude : Stored as an integer LAT_raw in the range 0…900

Actual latitude = LAT_raw / 10.0 °N

Longitude : Stored as an integer LON_raw in the range 0…3600

Actual longitude = LON_raw / 10.0 °E on a 0°–360° eastward basis

To convert into the more common –180°…+180° system (e.g. for GIS libraries):

LON_raw / 10.0 if lon > 180: lon = lon - 360


In [ ]:
# Apply the conversion element-wise
df['LAT'] = df['latitude']/10
df['LON'] = df['longitude']/10

# Use boolean indexing to select rows where 'LON' > 180 and update those values
df.loc[df['LON'] > 180, 'LON'] = df['LON'] - 360

df.head()

,cyclone_id,datetime,flag,intensity,latitude,longitude,num_data_lines,pressure,revision_date,storm_name,time_interval,track_sequence,wind,LAT,LON
0,CYC000001,1949011300,0,0,57,1399,49,1006,20110729.0,Carmen,6,1,0,5.7,139.9
1,CYC000001,1949011306,0,0,59,1393,49,1006,20110729.0,Carmen,6,2,0,5.9,139.3
2,CYC000001,1949011312,0,0,63,1387,49,1006,20110729.0,Carmen,6,3,0,6.3,138.7
3,CYC000001,1949011318,0,0,67,1380,49,1006,20110729.0,Carmen,6,4,0,6.7,138.0
4,CYC000001,1949011400,0,0,72,1373,49,1005,20110729.0,Carmen,6,5,0,7.2,137.3


In [ ]:
# prompt: group cyclone by id , take the mean of the lenght of groups

# Group the dataframe by 'id' and calculate the size of each group
grouped_sizes = df.groupby('cyclone_id').size()

# Calculate the mean of the group sizes
mean_group_length = grouped_sizes.mean()

print(f"The mean length of the groups (cyclones) is: {mean_group_length}")

The mean length of the groups (cyclones) is: 29.12575331458417


In [ ]:
from geographiclib.geodesic import Geodesic

def compute_stepwise(trajectory):
    """
    Given trajectory = [(lat1, lon1), (lat2, lon2), ...],
    returns a list of (distance_km, angle_from_equator_deg) between each pair.
    0° = east, 90° = north, etc.
    """
    geod = Geodesic.WGS84
    results = []
    for (lat1, lon1), (lat2, lon2) in zip(trajectory, trajectory[1:]):
        g = geod.Inverse(lat1, lon1, lat2, lon2)
        distance_km = g['s12'] / 1000.0
        # shift from "° from north" to "° from east"
        angle_deg = (g['azi1'] - 90) % 360
        results.append((distance_km, angle_deg))
    return results

# Example:
trajectory = [(23.4, 120.5), (23.6, 120.7), (23.9, 121.0)]
for i, (dist, ang) in enumerate(compute_stepwise(trajectory)):
    print(f"Step {i}->{i+1}: {dist:.2f} km, angle from equator = {ang:.1f}°")


Step 0->1: 30.13 km, angle from equator = 312.6°
Step 1->2: 45.16 km, angle from equator = 312.6°


In [ ]:
from geographiclib.geodesic import Geodesic

def compute_stepwise(trajectory):
    """
    Given trajectory = [(lat1, lon1), (lat2, lon2), ...],
    returns a list of (distance_km, angle_from_equator_deg) between each pair.
    0° = east, 90° = north, etc.
    """
    geod = Geodesic.WGS84
    results = []
    for (lat1, lon1), (lat2, lon2) in zip(trajectory, trajectory[1:]):
        g = geod.Inverse(lat1, lon1, lat2, lon2)
        distance_km = g['s12'] / 1000.0
        # shift from "° from north" to "° from east"
        angle_deg = (g['azi1'] - 90) % 360
        results.append((distance_km, angle_deg))
    return results

# Initialize new columns with default values (e.g., NaN)
df['step_distance_km'] = np.nan
df['step_bearing_deg'] = np.nan

# Group the DataFrame by 'tornado_id'
grouped_tornadoes = df.groupby('cyclone_id')

# Iterate through each tornado group
for tornado_id, tornado_df in grouped_tornadoes:
    # Extract the trajectory for the current tornado
    tornado_trajectory = list(zip(tornado_df['LAT'], tornado_df['LON']))

    # Check if the trajectory has at least two points to calculate steps
    if len(tornado_trajectory) > 1:
        # Compute the stepwise distances and bearings
        steps = compute_stepwise(tornado_trajectory)
        df.loc[tornado_df.index[1:], 'step_distance_km'] = [step[0] for step in steps]
        df.loc[tornado_df.index[1:], 'step_bearing_deg'] = [step[1] for step in steps]


In [ ]:
df.fillna(0, inplace=True)
df.head()

,cyclone_id,datetime,flag,intensity,latitude,longitude,num_data_lines,pressure,revision_date,storm_name,time_interval,track_sequence,wind,LAT,LON,step_distance_km,step_bearing_deg
0,CYC000001,1949011300,0,0,57,1399,49,1006,20110729.0,Carmen,6,1,0,5.7,139.9,0.000000,0.000000
1,CYC000001,1949011306,0,0,59,1393,49,1006,20110729.0,Carmen,6,2,0,5.9,139.3,70.035960,198.439056
2,CYC000001,1949011312,0,0,63,1387,49,1006,20110729.0,Carmen,6,3,0,6.3,138.7,79.798376,213.696187
3,CYC000001,1949011318,0,0,67,1380,49,1006,20110729.0,Carmen,6,4,0,6.7,138.0,89.171406,209.779649
4,CYC000001,1949011400,0,0,72,1373,49,1005,20110729.0,Carmen,6,5,0,7.2,137.3,95.085756,215.600100


In [ ]:
len(df['cyclone_id'].unique())

2489

In [ ]:
import numpy as np
from geographiclib.geodesic import Geodesic

geod = Geodesic.WGS84

def reconstruct_point(lat1, lon1, distance_km, angle_from_equator_deg):
    """
    Given start (lat1, lon1), distance (km), and angle_from_equator (deg, 0°=E),
    return the reconstructed (lat2, lon2).
    """
    # 1) Convert back to azimuth from north
    azi1 = (angle_from_equator_deg + 90) % 360

    # 2) Call Direct (distance in meters)
    g = geod.Direct(lat1, lon1, azi1, distance_km * 1000.0)
    return g['lat2'], g['lon2']

def compute_error(row):
    """
    Given a DataFrame row with lat1, lon1, orig lat2, lon2,
    plus computed step_distance_km, step_bearing_deg,
    returns the reconstruction error in meters.
    """
    lat1, lon1 = row['LAT_prev'], row['LON_prev']
    lat2, lon2 = row['LAT'],     row['LON']
    d_km, ang = row['step_distance_km'], row['step_bearing_deg']

    # skip if any missing
    if np.isnan(d_km) or np.isnan(ang):
        return np.nan

    # reconstruct
    lat2p, lon2p = reconstruct_point(lat1, lon1, d_km, ang)

    # measure error
    g_err = geod.Inverse(lat2, lon2, lat2p, lon2p)
    return g_err['s12']  # meters

# ——— Integrate into your DataFrame ———

# 1) Create shifted columns for the “previous” point
df['LAT_prev'] = df.groupby('cyclone_id')['LAT'].shift(1)
df['LON_prev'] = df.groupby('cyclone_id')['LON'].shift(1)

# 2) Compute errors row-wise
df['reconstruction_error_m'] = df.apply(compute_error, axis=1)
